<a href="https://colab.research.google.com/github/marcfpw/Independent-Study/blob/main/Independent_Project_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner -q
import keras_tuner

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 3.8 MB/s eta 0:00:00


In [3]:
# Prepare dataset to be used

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


In [10]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

build_model(keras_tuner.HyperParameters())

In [14]:
# Select tuner class to Run
# keras_tuner can be changed to .GridSearch, .RandomSearh, .Hyperband,.BayesianOptimization
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    seed=1,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

# Print summary of search space
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [15]:
# Start the search
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 10 Complete [00h 01m 25s]
val_accuracy: 0.964000016450882

Best val_accuracy So Far: 0.9721499979496002
Total elapsed time: 00h 12m 17s


In [16]:
# Get top 2 models
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
num_layers: 1
units_0: 512
activation: relu
dropout: False
lr: 0.0017611785596585996
Score: 0.9721499979496002

Trial 04 summary
Hyperparameters:
num_layers: 2
units_0: 384
activation: tanh
dropout: False
lr: 0.0012480675960619121
units_1: 288
units_2: 32
Score: 0.9681999981403351

Trial 01 summary
Hyperparameters:
num_layers: 3
units_0: 192
activation: relu
dropout: False
lr: 0.0013893292827173778
units_1: 32
units_2: 32
Score: 0.965800017118454

Trial 08 summary
Hyperparameters:
num_layers: 2
units_0: 480
activation: relu
dropout: True
lr: 0.00012165541012624621
units_1: 320
units_2: 128
Score: 0.965399980545044

Trial 09 summary
Hyperparameters:
num_layers: 1
units_0: 256
activation: tanh
dropout: False
lr: 0.0008960175671873151
units_1: 224
units_2: 64
Score: 0.964000016450882

Trial 07 summary
Hyperparameters:
num_layers: 3
units_0:

In [18]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 [==============================] - 18s 9ms/step - loss: 0.1832 - accuracy: 0.9444
